In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, roc_curve, auc, log_loss
from kmodes.kmodes import KModes

In [2]:
df = pd.read_csv('worker_master.csv')

In [3]:
df['worker'] = 1

In [4]:
df2 = pd.read_csv('loser_master.csv')

In [5]:
df2['worker'] = 0

In [6]:
master = df.append(df2, ignore_index = True)

In [7]:
master.shape

(3485, 7)

In [8]:
master = master.drop(['Unnamed: 0'], axis = 1)

In [9]:
master.dtypes

headline     object
hours         int64
jobs          int64
rate        float64
skills       object
worker        int64
dtype: object

In [10]:
master.shape

(3485, 6)

In [11]:
master = master.reset_index(drop = True)

In [12]:
master.iloc[57,:]

headline                                                  NaN
hours                                                     296
jobs                                                       12
rate                                                       23
skills      ['Creative Writing', 'English', 'English Gramm...
worker                                                      1
Name: 57, dtype: object

In [13]:
blanks = pd.isnull(master['headline'])

In [14]:
ind = blanks[blanks==True].index

In [15]:
master.iloc[ind,0] = ''

In [16]:
master.iloc[57,0]

''

In [17]:
master.shape

(3485, 6)

In [18]:
master = master.drop_duplicates()

In [19]:
master.shape

(3485, 6)

In [20]:
master['skills'] = master['skills'].map(lambda string: string.replace('[', ""))
master['skills'] = master['skills'].map(lambda string: string.replace(']', ""))
master['skills'] = master['skills'].map(lambda string: string.replace("'", ""))
master['skills'] = master['skills'].map(lambda string: string.replace('"', ''))
master['skills'] = master['skills'].map(lambda string: string.split(", "))

In [21]:
master['skills'] = master['skills'].map(lambda l: (l, [])[l == ['']])

In [22]:
master = master.reset_index(drop = True)

In [23]:
y = master.worker

In [24]:
type(y)

pandas.core.series.Series

In [25]:
X = master.drop(['worker', 'hours', 'jobs'], axis = 1)

In [26]:
X.skills = X.skills.map(lambda li: len(li))

In [27]:
fn = lambda row: len(row.headline) # define a function for the new column
X = X.assign(head_len = X.apply(fn, axis=1).values)

In [28]:
fn = lambda row: len(row.headline.split(' ')) # define a function for the new column
X = X.assign(head_num_words = X.apply(fn, axis=1).values)

In [29]:
min(X.head_len)

0

In [30]:
min(X.head_num_words)

1

In [31]:
X.tail()

,headline,rate,skills,head_len,head_num_words
3480,"Creative Writer: lesson plans, articles for bl...",35.0,3,66,9
3481,Financial Analyst,20.0,2,17,2
3482,Entry Level Python Programmer with a Focus on ...,30.0,5,70,12
3483,Full Stack Web Developer,19.0,10,24,4
3484,Graphic Designer/Illustrator,15.0,1,28,2


In [32]:
X = X.drop(['headline'], axis = 1)

In [33]:
rf = RandomForestClassifier(n_estimators = 500, max_depth = 5)
rfmodel = rf.fit(X, y)

In [34]:
rfmodel.score(X, y)

0.957532281205165

In [35]:
np.mean(cross_val_score(rfmodel, X, y, scoring = 'accuracy', cv = 4))

0.9575322707212

In [37]:
min(rfmodel.predict(X))

1

In [38]:
lr = LogisticRegression()
lrmodel = lr.fit(X, y)

In [40]:
lrmodel.score(X, y)

0.957532281205165

In [39]:
np.mean(cross_val_score(rfmodel, X, y, scoring = 'accuracy', cv = 4))

0.9575322707212